# Heart Attack Possibility

Christopher Mayorga | September 20, 2021

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
from scipy import stats

from acquire import get_df, get_info
from prepare import clean_heart_data


#visualize
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#train test split from sklearn
from sklearn.model_selection import train_test_split
#imputer from sklearn
from sklearn.impute import SimpleImputer

# to evaluate performance using rmse
from sklearn.metrics import mean_squared_log_error
from math import sqrt

# for tsa
import statsmodels.api as sm

# holt's linear trend model.
from statsmodels.tsa.api import Holt

# Decision Tree
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text

# Logistic Regression
from sklearn.linear_model import LogisticRegression

SyntaxError: invalid syntax (prepare.py, line 30)

## Acquire the Data

In [ ]:
# Bring in the heart.csv data from acquire.py
df = get_df()

In [ ]:
# Information Breakdown for Data
print(get_info(df))

Attribute Information
1) age
2) sex
3) chest pain type (4 values)
4) resting blood pressure
5) serum cholestoral in mg/dl
6) fasting blood sugar > 120 mg/dl
7) resting electrocardiographic results (values 0,1,2)
8) maximum heart rate achieved
9) exercise induced angina
10) oldpeak = ST depression induced by exercise relative to rest
11) the slope of the peak exercise ST segment
12) number of major vessels (0-3) colored by flourosopy
13) thal: 0 = normal; 1 = fixed defect; 2 = reversable defect
14) target: 0= less chance of heart attack 1= more chance of heart attack

Dataset is taken for learning purpose. Source of the data : https://archive.ics.uci.edu/ml/datasets/Heart+Disease

## Prepare/Clean Data

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
df = clean_heart_data(df)

In [ ]:
df.head()

## Exploring the Data

## Modeling

## Delivery

## Conlcusions